# Context-Enriched Retrieval in RAG
RAG은 외부 지식에서 관련 정보를 찾아와 AI의 응답 품질을 높이는 방식입니다. 하지만 기존의 검색 방식은 서로 떨어진 텍스트 조각들만 가져오기 때문에, 답변이 불완전하거나 맥락이 끊길 수 있습니다.

이 문제를 해결하기 위해, 문맥을 강화한 검색(Context-Enriched Retrieval) 방식을 소개합니다. 이 방법은 관련된 정보뿐만 아니라 주변 내용까지 함께 활용해, 보다 자연스럽고 완성도 높은 답변을 가능하게 합니다.

#### 이 노트북의 주요 단계는 다음과 같습니다:
- 데이터 불러오기: PDF 파일에서 텍스트를 추출합니다.
- 겹치는 구간으로 나누기: 문맥을 유지하기 위해 텍스트를 겹치는 조각들로 나눕니다.
- 임베딩 생성: 텍스트 조각들을 숫자 벡터로 변환합니다.
- 문맥 기반 검색: 관련된 텍스트와 그 주변 내용을 함께 검색합니다.
- 응답 생성: 검색된 문맥을 바탕으로 언어 모델이 답변을 생성합니다.
- 결과 평가: 생성된 답변의 정확도를 평가합니다.

## 환경 설정하기
필요한 라이브러리를 가져옵니다.

In [ ]:
import os
import numpy as np
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

API_KEY = os.environ.get('OPENAI_API_KEY')

# OpenAI API 클라이언트 설정하기

In [ ]:
from openai import OpenAI

client_openai = OpenAI(api_key = API_KEY)

## PDF 파일에서 텍스트 추출하기
RAG를 구현하려면 먼저 텍스트 데이터 소스가 필요합니다. 저는 gemini를 이용해 pdf에서 텍스트를 추출하는 방식을 사용합니다.  
만약 txt 형태로 파일이 존재한다면 `load_text_file` 함수를 사용하면됩니다.

In [ ]:
import google.generativeai as genai

def extract_text_from_pdf(pdf_path):
    # API 키 설정
    genai.configure(api_key=gemini_API_KEY)
    client = genai.GenerativeModel('gemini-2.0-flash-lite')

    # PDF 파일 업로드
    with open(pdf_path, "rb") as file:
        file_data = file.read()


    prompt = "Extract all text from the provided PDF file."
    response = client.generate_content([
        {"mime_type": "application/pdf", "data": file_data},
        prompt
    ],generation_config={
            "max_output_tokens": 8192  # 최대 출력 토큰 수 설정 (예: 8192 토큰, 약 24,000~32,000자)
    })
    return response.text

In [ ]:
# 이미 text 파일로 저장되어 있다면 load_text_file 함수를 사용하면 됩니다.
def load_text_file(pdf_path):

    # text 파일 로드
    with open(pdf_path, "r", encoding="utf-8") as txt_file:
        text = txt_file.read()

    return text

txt_path = "./data_creation/pdf_data/(1) 2024 달라지는 세금제도.txt"

extracted_text = load_text_file(txt_path)
print(extracted_text[:500])

## 추출된 텍스트 청크 분할
텍스트를 추출한 뒤에는 검색 정확도를 높이기 위해 조금씩 겹치도록 나눠서 작은 단위로 분할(chunk)합니다.  
이번 노트북에서는 여러 사이즈로 chunk를 분할합니다.

In [ ]:
def chunk_text(text, n, overlap):
    """
    주어진 텍스트를 n자 단위로, 일부가 겹치도록 chunking 합니다.

    Args:
    text (str): 청크할 텍스트입니다.
    n (int): 각 청크의 문자 수입니다.
    overlap (int): 청크 간 겹치는 문자 수입니다.

    Returns:
    List[str]: 청크된 텍스트 리스트입니다.
    """
    chunks = []  # 청크된 텍스트를 저장할 리스트
    
    # overlap만큼 겹치도록 text를 n의 길이로 chunking
    for i in range(0, len(text), n - overlap):
        chunks.append(text[i:i + n])

    return chunks  

## 추출된 텍스트 청크 분할
텍스트를 추출한 뒤에는 검색 정확도를 높이기 위해 조금씩 겹치도록 나눠서 작은 단위로 분할(chunk)합니다. 

In [ ]:
# 추출한 텍스트를 1000자씩, 200자가 겹치도록 나눕니다.
text_chunks = chunk_text(extracted_text, 1000, 200)

# 청크 개수 출력
print("Number of text chunks:", len(text_chunks))

# 첫 번째 청크 출력
print("\nFirst text chunk:")
print(text_chunks[0])

## 분할된 청크의 임베딩 생성
임베딩은 텍스트를 숫자 벡터로 변환하여 효율적인 유사도 검색을 가능하게 합니다.

In [ ]:
import torch
from sentence_transformers import SentenceTransformer
def create_embeddings(embedding_model, texts, device='cuda', batch_size=16):
    """
    SentenceTransformer 모델을 사용하여 지정된 텍스트에 대한 임베딩을 생성합니다.

    Args:
        embedding_model: 임베딩을 생성할 SentenceTransformer 모델입니다.
        texts (list): 임베딩을 생성할 입력 텍스트 리스트입니다.
        device (str): 모델을 실행할 장치 ('cuda' for GPU, 'cpu' for CPU).
        batch_size (int): 인코딩을 위한 배치 크기입니다.

    Returns:
        np.ndarray: 모델에 의해 생성된 임베딩입니다.
    """
    # 모델이 지정된 장치에 있는지 확인합니다.
    embedding_model = embedding_model.to(device)
    
    # 지정된 배치 크기로 임베딩을 생성합니다.
    embeddings = embedding_model.encode(
        texts,
        device=device,
        batch_size=batch_size,  # 메모리 사용량을 줄이기 위해 더 작은 배치 크기를 사용합니다.
        show_progress_bar=True  # 인코딩 진행 상태를 모니터링하기 위한 진행 표시줄을 표시합니다.
    )
    
    return embeddings

# GPU 사용 가능 여부를 확인합니다.
device = 'cuda' if torch.cuda.is_available() else 'mps'
print(f"Using device: {device}")

# 모델을 로드합니다.
model = "BAAI/bge-m3"
embedding_model = SentenceTransformer(model)

# 임베딩 생성
embeddings = create_embeddings(embedding_model, text_chunks, device=device, batch_size=4)

print(f"Generated {len(embeddings)} sentence embeddings.")

## Context-Aware Semantic Search 구현
chunk와 인접한 청크를 포함하도록 수정


In [ ]:
def cosine_similarity(vec1, vec2):
    """
    두 벡터 간의 코사인 유사도를 계산합니다.

    Args:
    vec1 (np.ndarray): 첫 번째 벡터입니다.
    vec2 (np.ndarray): 두 번째 벡터입니다.

    Returns:
    float: 두 벡터 간의 코사인 유사도입니다.
    """
    # 두 벡터의 내적을 계산하고 두 벡터의 크기의 곱으로 나눕니다.
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [ ]:
def context_enriched_search(query, text_chunks, embeddings, k=1, context_size=1):
    """
    가장 관련성이 높은 청크와 그 이웃 청크를 가져옵니다.

    Args:
    query (str): 검색 쿼리.
    text_chunks (List[str]): 텍스트 청크의 리스트.
    embeddings (List[dict]): 청크 임베딩의 리스트.
    k (int): 검색할 관련 청크의 수.
    context_size (int): 포함할 이웃 청크의 수.

    Returns:
    List[str]: 문맥 정보를 포함한 관련 텍스트 청크.
    """
    # 쿼리를 임베딩 벡터로 변환
    query_embedding =  create_embeddings(embedding_model, [query], device=device)[0]
    similarity_scores = []

    # 쿼리와 각 텍스트 청크 임베딩 간의 유사도 점수 계산
    for i, chunk_embedding in enumerate(embeddings):
        # 쿼리 임베딩과 현재 청크 임베딩 간의 코사인 유사도 계산
        similarity_score = cosine_similarity(np.array(query_embedding), np.array(chunk_embedding))
        # 인덱스와 유사도 점수를 튜플로 저장
        similarity_scores.append((i, similarity_score))


    # 유사도 점수에 따라 청크를 내림차순으로 정렬 (가장 높은 유사도 우선)
    similarity_scores.sort(key=lambda x: x[1], reverse=True)

    # 가장 관련성이 높은 청크의 인덱스 가져오기
    top_index = similarity_scores[0][0]

    # 문맥 포함 범위 정의
    # 0보다 작거나 text_chunks의 길이를 초과하지 않도록 보장
    start = max(0, top_index - context_size)
    end = min(len(text_chunks), top_index + context_size + 1)

    # 관련 청크와 그 이웃 문맥 청크 반환
    return [text_chunks[i] for i in range(start, end)]

## 테스트

In [ ]:
import pandas as pd
# 평가 데이터 로드하기
df = pd.read_csv('./data_creation/rag_val_new_post.csv')
df.head()

In [ ]:
# 평가 데이터에서 첫 번째 쿼리 추출
query = df['query'][0]

# 가장 관련성이 높은 청크와 그 이웃 청크를 가져와서 context로 활용
# 매개변수:
# - query: 검색할 질문
# - text_chunks: PDF에서 추출한 텍스트 청크
# - response.data: 텍스트 청크의 임베딩
# - k=1: 가장 일치하는 항목 반환
# - context_size=1: 문맥을 위해 가장 일치하는 항목 전후로 1개의 청크 포함
top_chunks = context_enriched_search(query, text_chunks, embeddings, k=1, context_size=1)

# 쿼리 출력
print("쿼리:", query)

# 각 검색된 청크를 제목과 구분선과 함께 출력
for i, chunk in enumerate(top_chunks):
    print(f"문맥 {i + 1}:\n{chunk}\n=====================================")

## 검색된 청크를 기반으로 response 생성하기

In [ ]:
def generate_response(system_prompt, user_message ,model_name='gpt-4.1-nano'):
    
    response = client_openai.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": system_prompt},# Define the system prompt for the AI assistant
            {"role": "user", "content": user_message}
        ],
        temperature=0.1,
        top_p=0.9,
        max_tokens=1024,
    )
    # print(response.choices[0].message.content)

    return response

# 사용자 프롬프트
user_prompt = "\n".join([f"Context {i + 1}:\n{chunk}\n=====================================\n" for i, chunk in enumerate(top_chunks)])
user_prompt = f"{user_prompt}\nQuestion: {query}"

# 시스템 프롬프트
system_prompt = "당신은 제공된 Context에 기반하여 답변하는 AI 어시스턴트입니다. 답변이 컨텍스트에서 직접 도출될 수 없는 경우, 다음 문장을 사용하세요: '해당 질문에 답변할 충분한 정보가 없습니다.'"

# 응답 생성
ai_response = generate_response(system_prompt, user_prompt)

In [ ]:
print(ai_response.choices[0].message.content)

## 생성 응답 평가하기
생성된 답변과 예상 답변을 비교하여 평가합니다. 답변 평가시에는 LLM을 사용합니다.

In [ ]:
# 평가를 위한 시스템 프롬프트
evaluate_system_prompt = "You are an intelligent evaluation system tasked with assessing the AI assistant's responses. If the AI assistant's response is very close to the true response, assign a score of 1. If the response is incorrect or unsatisfactory in relation to the true response, assign a score of 0. If the response is partially aligned with the true response, assign a score of 0.5."

# 사용자 쿼리, 생성된 답변, 참고(정답) 답변, 평가 시스템 프롬프트를 결합하여 평가 프롬프트 생성
evaluation_prompt = f"User Query: {query}\nAI Response:\n{ai_response.choices[0].message.content}\nTrue Response: {df['generation_gt'][0]}\n{evaluate_system_prompt}"

# 평가 응답 생성
evaluation_response = generate_response(evaluate_system_prompt, evaluation_prompt,'gpt-4.1-mini')

# 평가 응답 출력
print(evaluation_response.choices[0].message.content)

# 전체 추론 평가

In [ ]:
# 생성된 답변과 실제 답변을 비교하여 점수 부여
result = []

for i in range(len(df)):
    query = df['query'][i]
    top_chunks = context_enriched_search(query, text_chunks, embeddings, k=1, context_size=1)

    # user 프롬프트
    user_prompt = "\n".join([f"Context {i + 1}:\n{chunk}\n=====================================\n" for i, chunk in enumerate(top_chunks)])
    user_prompt = f"{user_prompt}\n질문: {query}"

    # system 프롬프트
    system_prompt = "당신은 제공된 Context에 기반하여 답변하는 AI 어시스턴트입니다. 답변이 컨텍스트에서 직접 도출될 수 없는 경우, 다음 문장을 사용하세요: '해당 질문에 답변할 충분한 정보가 없습니다.'"

    # response 생성
    ai_response = generate_response(system_prompt, user_prompt, 'gpt-4.1-mini')

    print(ai_response.choices[0].message.content)

    # 평가 시스템 프롬프트
    evaluate_system_prompt = "You are an intelligent evaluation system tasked with assessing the AI assistant's responses. If the AI assistant's response is very close to the true response, assign a score of 1. If the response is incorrect or unsatisfactory in relation to the true response, assign a score of 0. If the response is partially aligned with the true response, assign a score of 0.5. \n The answer should only output numbers and should not output any words."

    # 사용자 쿼리, 생성된 답변, 참고(정답) 답변, 평가 시스템 프롬프트를 결합하여 평가 프롬프트 생성
    evaluation_prompt = f"User Query: {query}\nAI Response:\n{ai_response.choices[0].message.content}\nTrue Response: {df['generation_gt'][i]}\n{evaluate_system_prompt}"

    # 평가 응답 생성
    evaluation_response = generate_response(evaluate_system_prompt, evaluation_prompt,'gpt-4.1-mini')

    # 평가 응답 출력
    print(evaluation_response.choices[0].message.content)
    result.append(evaluation_response.choices[0].message.content)

In [ ]:
import numpy as np
result = [float(x.replace('\n','').replace('Score: ',''))for x in result]
np.average(result)